In [1]:
import pandas as pd
from tqdm import tqdm
from glob import glob
import numpy as np

In [119]:
cl = [e[3:] for e in "223SLHC,224HST,225HCT,226MCV,227MCH,228MCHC,234SLTC,246SLBC,233RDW-CV,indate".split(",")]
cl

['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'SLTC', 'SLBC', 'RDW-CV', 'ate']

In [95]:
# for name in ['TBM']:
#     files = glob(f"../datasets/{name}/*")
#     df_list = [pd.read_csv(e).set_index("PID")[cl] for e in files]
#     df = pd.concat(df_list)
#     df.to_csv(f"../datasets/TONG_HOP/{name}.csv")

In [96]:
files = glob(f"../datasets/TONG_HOP/*")
df_list = [pd.read_csv(e).set_index("PID") for e in files]

In [97]:
files

['../datasets/TONG_HOP\\DIEN_DI.csv',
 '../datasets/TONG_HOP\\DI_TRUYEN.csv',
 '../datasets/TONG_HOP\\pid.csv',
 '../datasets/TONG_HOP\\SINH_HOA.csv',
 '../datasets/TONG_HOP\\TBM.csv']

In [98]:
pid_df = df_list[2]
pid_df.index = pid_df.index.astype(int)

tbm_df = df_list[-1].assign(inyear=df_list[-1].indate.str.split('-').str[0])
sinh_hoa_df = df_list[-2].assign(inyear=df_list[-2].indate.str.split('-').str[0])
sinh_hoa_df = sinh_hoa_df[['FE', 'FERRITIN', 'inyear']]
dien_di_df  = df_list[0].assign(inyear=df_list[0].indate.str.split('-').str[0])
dien_di_df = dien_di_df[['HBA1', 'HBE', 'HBF', 'HBA2', 'HBH', 'HBB', 'inyear']]

In [99]:
tong_hop_df = pid_df.merge(tbm_df, how='left', left_index=True, right_index=True)
tong_hop_df = tong_hop_df.merge(sinh_hoa_df, how='outer', left_on=['PID', 'inyear'], right_on=['PID', 'inyear'])
tong_hop_df = tong_hop_df.merge(dien_di_df, how='outer', left_on=['PID', 'inyear'], right_on=['PID', 'inyear'])
tong_hop_df

,SLHC,HST,HCT,MCV,MCH,MCHC,SLTC,SLBC,RDW-CV,indate,inyear,FE,FERRITIN,HBA1,HBE,HBF,HBA2,HBH,HBB
PID,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123456,5.32,118.0,0.383,71.9,22.2,309.0,264.0,7.12,NaN,2016-08-29,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123456,2.00,2.0,2.000,2.0,2.0,2.0,2.0,2.00,2,2019-06-14,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000544,2.93,64.0,0.228,77.9,22.0,282.0,167.0,9.81,NaN,2016-04-23,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008410,3.87,142.0,0.421,108.8,36.8,338.0,113.0,3.98,14.0,2018-10-20,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920045023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,70.8,29.2,NaN,NaN,NaN,NaN
1920054399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,98.3,NaN,NaN,1.7,NaN,NaN
1920054400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,98.5,NaN,NaN,1.5,NaN,NaN


In [100]:
di_truyen_df = df_list[1]
di_truyen_df.index = di_truyen_df.index.astype(int)
di_truyen_df = di_truyen_df[~di_truyen_df.index.duplicated(keep='first')]

In [101]:
from mapping import mp, alpha_names, beta_names, pattern
def process_header(df):
    target_lb = list(pattern.keys())

    for idx, e in enumerate(df.columns.values):
        for k in target_lb:
            if k.lower() in str(e).lower():
                df.columns.values[idx] = f'{k}_{idx}'
                break
    df.columns = df.columns.values

In [102]:
process_header(di_truyen_df)

In [103]:
keep_columns = [e for e in di_truyen_df.columns.values if '_' in e]

In [104]:
keep_idx = di_truyen_df[keep_columns].dropna(how='all').index

In [105]:
result_df = di_truyen_df[['indate'] + keep_columns].loc[keep_idx]

In [106]:
result_df = result_df.replace(regex='((Dương)|(Duong)|(DƯƠNG)|(duong)|(DUONG))+', value=-2)

In [107]:
lb = np.any(result_df == -2, axis=1).astype(int)

In [108]:
result_df = result_df.assign(label = lb)

In [109]:
pt_df = result_df[['indate', 'label']]

In [110]:
pt_df

,indate,label
PID,,
7008235,﻿2013-03-26,0
8004477,2013-03-26,0
9002987,2013-03-08,1
9004809,2013-03-11,0
9006069,2013-03-12,0
...,...,...
2020109668,2020-01-31,1
2020109669,2020-01-31,1
2020109670,2020-01-31,1


In [111]:
tong_hop_df = tong_hop_df.merge(pt_df, how='left', left_on=['PID'], right_on=['PID'])


In [127]:
mask_df = tong_hop_df[['SLHC', 'HST', 'HCT', 'MCV', 'MCH', 'MCHC', 'SLTC', 'SLBC', 'RDW-CV']]
tong_hop_df[np.any(mask_df, axis=1)].to_csv('tong_hop.csv')

In [114]:
tong_hop_df